In [1]:
from tqdm import tqdm
import pdb
import limix
import pickle as pkl
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from numpy import asarray
import xarray as xr
from copy import deepcopy
import os
from os.path import join
import simplejson as json
import hashlib
import pandas as pd
import pdb
from glob import glob
import h5py
import re
from plot import plot_poisson_ordered, plot_distplots, plot_distplot
from util import get_amelie_selection
import h5py
import dask
import dask.dataframe
import dask.array
from urllib.request import urlopen
try:
    import ipfsapi
    has_ipfs = True
except ModuleNotFoundError:
    has_ipfs = False


if has_ipfs:
    try:
        ipfs = ipfsapi.connect('127.0.0.1', 5001)
    except Exception:
        has_ipfs = False

only_hash = True

# Load data (Amelie's and Hannah's)

In [2]:
if os.path.exists("data/traits-kinship-hannah-amelie.pkl"):
    data = pkl.load(open("data/traits-kinship-hannah-amelie.pkl", "rb"))
else:
    url = "http://ipfs.io/ipfs/QmXtFuoA3JTkhhpUpqh4dyu3FJLUG7DWAmXwUk1N2TMbAh"
    data = pkl.load(urlopen(url))
    
dst_folder = "3"
if not os.path.exists(dst_folder):
    os.mkdir(dst_folder)
if not os.path.exists(join(dst_folder, 'phenotype')):
    os.mkdir(join(dst_folder, 'phenotype'))
if not os.path.exists(join(dst_folder, 'kinship')):
    os.mkdir(join(dst_folder, 'kinship'))

In [3]:
def isbernoulli(x):
    x = np.asarray(x, float)
    u = np.unique(x)
    if len(u) == 2:
        return True
    return False

def get_bernoulli(x):
    x = np.asarray(x, float)
    u = np.unique(x)
    i0 = x == u[0]
    i1 = x == u[1]
    x[i0] = 0.0
    x[i1] = 1.0
    return x

def isdiscrete(x):
    x = np.asarray(x, float)
    ok = np.isfinite(x)
    return all(x[ok] == np.asarray(x[ok], int))

def get_poisson(x):
    x = np.asarray(x, float)
    mi = min(x)    
    if mi < 0:
        x += -mi
    return x

def isnumber(x):
    try:
        np.asarray(x, float)
    except ValueError:
        return False
    return True

In [12]:
def analysis_QTL(G, pheno, kinship, pheno_name, lik, pheno_norm):
    if pheno_name != "Avoidances31_40":
        return
    import pdb
    common_samples = set(kinship.index.values).intersection(set(pheno.index.values))
    common_samples = np.asarray(list(common_samples))
    
    pheno = pheno.reindex(common_samples).copy()
    kinship = kinship.reindex(index=common_samples).copy()
    kinship = kinship.reindex(columns=common_samples).copy()
    
    assert(all(kinship.index.values == kinship.columns.values))
    assert(all(kinship.index.values == pheno.index.values))

    ok = np.isfinite(pheno.loc[:, pheno_name])
    common_samples = common_samples[ok]
    
    pheno = pheno.reindex(common_samples).copy()
    kinship = kinship.reindex(index=common_samples).copy()
    kinship = kinship.reindex(columns=common_samples).copy()
    
    assert(all(kinship.index.values == kinship.columns.values))
    assert(all(kinship.index.values == pheno.index.values))
    
    if not all(np.isfinite(pheno[pheno_name])):
        raise ValueError("not all finite: {}".format(pheno_name))
    
    y = pheno_norm(pheno.loc[:, pheno_name])
    
    phenotype_series = pd.Series(y, pheno.loc[:, pheno_name].index.values)
    pdb.set_trace()
    obj = pkl.dumps(phenotype_series)
    m = hashlib.sha1()
    m.update(obj)
    phenotype_hex = m.hexdigest()
    with open(join(dst_folder, "phenotype", phenotype_hex + ".series.pkl"), "wb") as f:
        f.write(obj)
    
    obj = pkl.dumps(kinship)
    m = hashlib.sha1()
    m.update(obj)
    kinship_hex = m.hexdigest()
    with open(join(dst_folder, "kinship", kinship_hex + ".dataframe.pkl"), "wb") as f:
        f.write(obj)

    try:
        call = "limix.qtl.scan(G, y, lik, kinship, verbose=True)"
        model = limix.qtl.scan(G, y, lik, kinship, verbose=True)
    except ValueError as e:
        print(e)
        print("Pheno name: {}".format(pheno_name))
        return None

    return dict(pv=list(asarray(model.variant_pvalues)), lik=lik, call=call, phenotype=phenotype_hex, kinship=kinship_hex,
                null_covariate_effsizes=list(asarray(model.null_covariate_effsizes)),
                variant_effsizes=list(asarray(model.variant_effsizes)), variant_effsizes_se=list(asarray(model.variant_effsizes_se)))

In [13]:
f = h5py.File("arrayexpress/HS.hdf5", "r")
samples = np.asarray(f["/imputed_genotypes/row_header/rat"].value)
samples = [i.decode() for i in samples]
f.close()

In [14]:
data0 = deepcopy(data)
remove = []
for name in data0['measures'].columns.values:
    if isnumber(data0['measures'].loc[:, name]) and isdiscrete(data0['measures'].loc[:, name]):
        data0['measures'].loc[:, name] = get_poisson(data0['measures'].loc[:, name])
    else:
        remove.append(name)
for name in remove:
    del data0['measures'][name]

In [15]:
Gs = []
for i in tqdm(range(1, 22)):
#     G = xr.open_dataset("/Users/horta/arrayexpress/HS.hdf5", "/imputed_genotypes/chr{}".format(i))["array"]
    G = xr.open_dataarray("arrayexpress/HS.hdf5", "/imputed_genotypes/chr{}".format(i),
                          chunks=1000)
    G = G.rename({G.dims[0]: "snps", G.dims[1]: "samples"})
    Gs.append(G)

100%|██████████| 21/21 [00:01<00:00, 13.29it/s]


In [16]:
G = xr.concat(Gs, dim="snps").T
G['samples'] = samples
G['snps'] = range(G.shape[1])

In [ ]:
for name in data0['measures'].columns.values:
#     if name != "Avoidances21_30":
#         continue
    print("Processing: {}".format(name))
    dst_file = join(dst_folder, "scan_measures_poisson_" + name + ".json")
#     if os.path.exists(dst_file) or os.path.exists(dst_file + ".failed"):
#         continue
    r = analysis_QTL(G, data0['measures'], data0['kinship'], name, 'poisson', lambda x: x)
#     if r is None:
#         open(dst_file + ".failed", "w").write("")
#     else:
#         json.dump(r, open(dst_file, "w"))

Processing: Boli_tot
Processing: Distance0_30
Processing: Distance10_15
Processing: Distance15_20
Processing: Distance20_25
Processing: Distance25
Processing: Distance25_30
Processing: Distance5
Processing: Distance5_10
Processing: Rearing0_30
Processing: Rearing10_15
Processing: Rearing15_20
Processing: Rearing20_25
Processing: Rearing25
Processing: Rearing5
Processing: Rearing5_10
Processing: ALP
Processing: Chloride
Processing: Sodium
Processing: BW_week9
Processing: Has1kidney
Processing: PLT
Processing: AA_IL_nb
Processing: AA_IL_score
Processing: AA_nb
Processing: AA_score
Processing: IL_nb
Processing: IL_score
Processing: Avoidances1_10
Processing: Avoidances1_20
Processing: Avoidances1_40
Processing: Avoidances1_5
Processing: Avoidances11_20
Processing: Avoidances21_30
Processing: Avoidances31_40
> <ipython-input-12-bfad2bc1e652>(32)analysis_QTL()
-> obj = pkl.dumps(phenotype_series)


(Pdb)  !kinship


             Outbred1306  Outbred1736  Outbred1527  Outbred585  Outbred1206  \
Outbred1306     6.285775     5.037059     5.260183    5.047880     5.045315   
Outbred1736     5.037059     6.136208     5.384032    4.949385     4.941058   
Outbred1527     5.260183     5.384032     6.384055    5.096398     5.128256   
Outbred585      5.047880     4.949385     5.096398    6.192159     4.808440   
Outbred1206     5.045315     4.941058     5.128256    4.808440     6.097120   
Outbred496      4.967618     5.229495     5.270797    4.869244     4.896030   
Outbred952      5.012510     5.121703     5.268359    4.843483     4.964771   
Outbred242      4.938836     4.855036     4.990796    4.836580     4.840440   
Outbred1677     5.341648     5.099375     5.197439    4.973386     5.077538   
Outbred1742     4.890509     5.044671     5.115463    4.757521     4.839160   
Outbred757      5.007122     4.961937     5.107505    5.047463     4.859286   
Outbred175      5.058167     5.034456     5.139683  

In [10]:
f.close()

In [ ]:
type(G)